# Execution in Webserver Root Folder
Detects a suspicious program execution in a web service root folder (filter out false positives)

## Rule Content
```
- title: Execution in Webserver Root Folder
  id: 35efb964-e6a5-47ad-bbcd-19661854018d
  status: experimental
  description: Detects a suspicious program execution in a web service root folder
    (filter out false positives)
  author: Florian Roth
  tags:
  - attack.persistence
  - attack.t1100
  logsource:
    category: process_creation
    product: windows
    service: null
  detection:
    selection:
      Image:
      - '*\wwwroot\\*'
      - '*\wmpub\\*'
      - '*\htdocs\\*'
    filter:
      Image:
      - '*bin\\*'
      - '*\Tools\\*'
      - '*\SMSComponent\\*'
      ParentImage:
      - '*\services.exe'
    condition: selection and not filter
  fields:
  - CommandLine
  - ParentCommandLine
  falsepositives:
  - Various applications
  - Tools that include ping or nslookup command invocations
  level: medium

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(process_path.keyword:(*\\wwwroot\\* OR *\\wmpub\\* OR *\\htdocs\\*) AND (NOT (process_path.keyword:(*bin\\* OR *\\Tools\\* OR *\\SMSComponent\\*) AND process_parent_path.keyword:(*\\services.exe))))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()